# "Naive Bayes Algorithm for Detecting Spam Messages"

> "I code a multinomial naive bayes algorithm, step-by-step, in order to identify messages as spam or non-spam."

- author: Migs Germar
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, pandas, numpy, matplotlib, seaborn, altair]
- hide: true
- search_exclude: true
- image: images/spam-unsplash-hannes_johnson.jpg

<center><img src = "https://miguelahg.github.io/mahg-data-science/images/spam-unsplash-hannes_johnson.jpg" alt = ""></center>

<center><a href = "https://unsplash.com/photos/mRgffV3Hc6c">Unsplash | Hannes Johnson</a></center>

# Introduction

The Multinomial Naive Bayes Algorithm is a machine learning algorithm based on the Bayes Theorem. It calculates the probability that an event $B$ occurred given that event $A$ occurred. Thus, it is usually used in classification problems. (Vadapalli, 2021)

In this project, we will use the algorithm to determine the probability that a message is spam given its contents. We will then use this probability to decide whether to treat new messages as spam or not. For example, if the probability of being spam is over 50%, then we may treat the message as spam.

Identifying spam is important in the Philippines because phishing campaigns went up by 200% after the pandemic began (Devanesan, 2020), and a telecommunications provider recently had to block around 71 million spam messages (Yap, 2021). Such messages may attempt to steal personal information, steal money from an account, or install malware (FTC, 2020). Thus, machine learning can be a very helpful tool in preventing such harm from occurring.

Though the algorithm can be easily implemented using existing functions such as those in the [`scikit-learn` package](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes), I will manually code the algorithm step-by-step in order to explain the mathematical intuition behind it.

> Note: I wrote this notebook by following a guided project on the [Dataquest](https://www.dataquest.io/) platform, specifically the [Guided Project: Building a Spam Filter with Naive Bayes](https://app.dataquest.io/c/74/m/433/guided-project%3A-building-a-spam-filter-with-naive-bayes/1/exploring-the-dataset) The general project flow came from Dataquest. The mathematical explanations are also based on what I learned from Dataquest.

# Preparations

Below are the packages necessary for this project.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# The Dataset

We will use the SMS Spam Collection Dataset by Almeida and Hidalgo in 2012. It can be downloaded from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).



In [6]:
#collapse-hide
sms_df = pd.read_csv(
    "./private/Naive-Bayes-Files/SMSSpamCollection",
    # Tab-separated
    sep = "\t",
    header = None,
    names = ["label", "sms"]
)

sms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   object
 1   sms     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


The dataset has 2 columns and 5572 rows.

- The `label` column contains "ham" if the message is legitimate, or "spam" if it is spam.
- The `sms` column contains individual SMS messages.

For example, below are the first 5 rows of the dataset.

In [7]:
#collapse-hide
sms_df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Training and Testing Sets

The messages will be split into two sets. The training set, comprising 80% of the total data, will be used to train the Naive Bayes Algorithm. The testing set, with 20% of the total data, will be used to test the model's accuracy.

First, however, let us calculate what percentage of the messages in the dataset are spam.

In [19]:
#collapse-hide
spam_perc = sms_df["label"].eq("spam").sum() / sms_df.shape[0] * 100
print(f"Percentage of spam messages: {spam_perc:.2f}%")

Percentage of spam messages: 13.41%


Only 13% of the messages are spam. Therefore, spam and non-spam messages are not equally represented in this dataset, and this may be problematic. However, this is all the data we have, so the best we can do is to ensure that both the training and testing sets have around 13% of their messages as spam.

This is an example of *proportional stratified sampling*. We first separate the data into two strata (spam and non-spam). We then take 80% of the messages from each strata as the training set. The remaining 20% of each strata is set aside for the testing set.

This has been done with the code below.

In [30]:
#collapse-hide
# Note: I could have used `train_test_split` from sklearn, but I coded this manually for the sake of grasping the logic.
split_lists = {
    "training": [],
    "testing": [],
}

# Stratify the dataset
for label in "spam", "ham":
    stratum = sms_df.loc[sms_df["label"] == label]

    train_part = stratum.sample(
        # Sample 80% of the data points
        frac = 0.8,
        random_state = 1,
    )

    # The other 20% that were not sampled go to the testing set.
    test_part = stratum.loc[~stratum.index.isin(train_part.index)]

    split_lists["training"].append(train_part)
    split_lists["testing"].append(test_part)

split_dfs = pd.Series(dtype = "object")
for key in split_lists:
    # Concatenate spam and non-spam parts into one DataFrame.
    set_df = pd.concat(split_lists[key]).reset_index()
    split_dfs[key] = set_df

    perc_spam = set_df.label.eq('spam').sum() / set_df.shape[0] * 100

    print(f"Number of rows in {key} set: {set_df.shape[0]}")
    print(f"Percentage of {key} messages that are spam: {perc_spam:.2f}%")

Number of rows in training set: 4458
Percentage of training messages that are spam: 13.41%
Number of rows in testing set: 1114
Percentage of testing messages that are spam: 13.38%


We can see that the percentage of spam messages is roughly the same across the two sets. This will help the accuracy of the model later on.

Now, the two sets will be further split into `X` and `y`. `y` refers to the **target**, or the variable that we are trying to predict. In this case, we are trying to predict whether a message is spam or non-spam, so the "label" column is the target:

In [34]:
#collapse-hide
sms_df.label.head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object

On the other hand, `X` refers to the **features**, which are information used to predict the target. We only have one feature column as of now, which is the "sms" column.

In [35]:
#collapse-hide
sms_df.sms.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: sms, dtype: object


Thus, we end up with four final objects:

- `X_train`: The messages in the training data.
- `X_test`: The messages in the testing data.
- `y_train`: The labels in the training data. These correspond to `X_train`.
- `y_test`: The labels in the testing data. These correspond to `X_test`.

In [33]:
#collapse-show
# The four objects listed above.
X_train = split_dfs.training[["sms"]]
X_test = split_dfs.testing[["sms"]]
y_train = split_dfs.training["label"]
y_test = split_dfs.testing["label"]

# The Algorithm

Now, let's discuss the multinomial naive bayes algorithm itself. As mentioned earlier, it is based on the Bayes Theorem. Given two events $A$ and $B$, we can use the theorem to determine the probability that $B$ happened given that $A$ happened. This probability is written as $P(B|A)$.

$P(B|A) = \frac{P(B) \cdot P(A|B)}{\Sigma_{i = 1}^n (P(B_i) \cdot P(A|B_i))}$

In this case, $B_1$ is the event that the message is non-spam, and $B_2$ is the event that it is spam. $B$ can refer to either $B_1$ or $B_2$, depending on which probability we want to calculate. Also, $A$ refers to the specific contents of one message.

In order to make things clearer, let us say that $Spam$ is the event that the message is spam, and $Spam^C$ is the event that the message is non-spam.

Then, let us expand event $A$ (the message itself) in order to consider the individual words inside it. For example, the first word in a message can be labeled $w_1$. If we have a total of $n$ words, then the words can be labeled as $w_1, w_2, \dots , w_n$.

Thus, we can rewrite the equation. Here is the probability of a given message being spam:

$P(Spam|w_1, w_2, \dots , w_n) = \frac{P(Spam) \cdot P(w_1, w_2, \dots , w_n|Spam)}{\Sigma_{i = 1}^n (P(B_i) \cdot P(w_1, w_2, \dots , w_n|B_i))}$

Here is the probability of a given message being non-spam:

$P(Spam^C|w_1, w_2, \dots , w_n) = \frac{P(Spam^C) \cdot P(w_1, w_2, \dots , w_n|Spam^C)}{\Sigma_{i = 1}^n (P(B_i) \cdot P(w_1, w_2, \dots , w_n|B_i))}$

Notice that the denominators are the same. Since we only want to compare these two probabilities, we can skip calculating the denominator and just calculate the numerators. We can thus rewrite the equation as follows. Note that the $\propto$ symbol is used instead of $=$ because the two quantities are not equal but directly proportional.

$P(Spam|w_1, w_2, \dots , w_n) \propto P(Spam) \cdot P(w_1, w_2, \dots , w_n|Spam)$

The first factor, $P(Spam)$, is easy to find, as it is simply the number of spam messages divided by the total number of messages. However, $P(w_1, w_2, \dots , w_n|Spam)$ needs to be further expanded.

If we make the assumption that the probability of each word is independent of the probability of the other words, we can use the multiplication rule. The assumption of independence is what makes the algorithm "naive," as it usually doesn't hold true in reality. However, the algorithm is still useful for predictions despite this.

$P(Spam) \cdot P(w_1, w_2, \dots , w_n|Spam) \\ = P(Spam) \cdot P(w_1 \cap w_2 \cap \dots \cap w_n | Spam) \\ = P(Spam) \cdot P(w_1|Spam) \cdot P(w_2|Spam) \cdot \dots \cdot P(w_n|Spam)$

Note that we still have to find the probability of each word given $Spam$ because we assume that the presence of each word is dependent on $Spam$.

Thus, at this point, the final formula is:

$P(Spam|w_1, w_2, \dots , w_n) \propto P(Spam) \cdot \Pi_{i=1}^n P(w_i|Spam)$